# Status Update

(By the way, I'm writing this in Markdown, i.e. I typed this, and then changed the Code option in the control bar at the top of the page to the Markdown option.)

Yesterday (August 7) I worked on getting a surface where the geodesics intersected the meridian at 0.2 * pi (or 0.100 * tau, which is why the file is named Plotting_dot100tau_part2.ipynb). Using a degree 10 polynomial didn't work out well, as the computer kept adding little bumps in the function to adjust the angles, instead of finding a smooth shape. I changed to a degree 4 polynomial, which gave me a smooth shape and pretty small deviation; see the plot below.

In [1]:
old_data = [(0, 2.59759769147076),
 (4/5, 3.93854551237998),
 (8/5, 4.43146572806194),
 (12/5, 4.51594411111890),
 (16/5, 3.99122898960989),
 (4, 3.80689679534281),
 (24/5, 3.41762882390323),
 (28/5, 2.59805658601637),
 (32/5, 2.35228009244850),
 (36/5, 1.95642349521190),
 (8, 1.23478792547764)]
#Why did I use 11 data points when a quartic can be described by 5? I don't know.

In [2]:
def line_plot_points(function1, u_bound, density = 100, color = 'red'):
    '''Plots points on a curve'''
    A = []
    for i in range(density+1):
        u = u_bound[0] * (density - i) + u_bound[1] * i
        u = u/density
        temp = function1(u)
#        print( type(temp[0]) )
#        print(type(1.0))
        if type(temp[0]) is sage.rings.complex_number.ComplexNumber:
            break
        A.append( temp )
#        print(u, function(u), type(function(u)[0]))
#    print(A)
    return point3d(A, color = color, size = 20) 


def surface_plot(function2, u_bound, v_bound, density = 50):
    '''Plots a surface'''
    A = []
    for i in range(density+1):
        u = u_bound[0] * (density - i) + u_bound[1] * i
        u = u/density
        for j in range(density+1):
            v = v_bound[0] * (density - j) + v_bound[1] * j
            v = v/density
            A.append( function2(u,v) )
    return point3d(A, size = 20)




tau = 2*pi


def find_angle2(target, increment = 0.01):
    import math, random
    '''Finds the angle of intersection between the geodesic and the meridian'''
    upper = low 
    while not math.isnan(g(upper)) and upper <= high:
#        Find the upper bound on where g is defined.
        upper += (high-low)/100
    upper -= (high-low)/100
    #Now find the root 
    try:
        u_0 = find_root(lambda x: g(x)-target, low, upper)
    except:
        print("Runtime error")
        return 1000
#    print(n(u_0))
    u_1 = u_0 + (high-low)*increment
    v_0 = g(u_0)
    v_1 = g(u_1)
    meridian_vec = vector([f_prime(u_0)* cos(v_0), f_prime(u_0)* sin(v_0),1])
    geodesic_vec = vector([f(u_1) * cos(v_1), f(u_1) * sin(v_1), u_1]) - vector([f(u_0) * cos(v_0), f(u_0) * sin(v_0), u_0])
#    print(meridian,geodesic)
    dotproduct = meridian_vec.dot_product(geodesic_vec)
    ratio = dotproduct/(norm(meridian_vec) * norm(geodesic_vec))
    return n(arccos(ratio)/tau)
#   e.g. if the actual angle is pi/2 = tau/4, this returns 0.25



def deviation(numbers, target_mean):
    '''Square root of the average squared difference from a target'''
    the_sum = 0
    for x in numbers:
        the_sum += (x - target_mean)^2
    return n(sqrt(the_sum/len(numbers)))

def standard_deviation(numbers):
    '''Does NOT use sample variance'''
    the_sum = 0
    for x in numbers:
        the_sum += x
    the_mean = the_sum/len(numbers)
    return deviation(numbers, the_mean)
    
def should_stop():
    '''Returns true if the word "stop" is in the file often_overwritten.txt'''
    f = open("often_overwritten.txt","r")
    stop = "stop" in f.read().lower()
# i.e. the variable stop records whether the word "stop" is in the file, ignoring case
    f.close()
    return stop

In [3]:



#This method plots everything.


import pdb

data = old_data

#Find the polynomial that fits the data

var('a_10,a_9,a_8,a_7,a_6,a_5,a_4,a_3,a_2,a_1,a_0,t')

#model(t)= a_10*t^10 + a_9*t^9 + a_8*t^8 + a_7*t^7 + a_6*t^6 + a_5*t^5 +a_4*t^4 + a_3*t^3 + a_2*t^2 + a_1*t + a_0

model(t)=  a_4*t^4 + a_3*t^3 + a_2*t^2 + a_1*t + a_0

fit=find_fit(data,model)






var('t u v s')


def f(t):
    return model.subs(fit)(t)

def f_prime(t):
    return derivative(model.subs(fit))(t)
    
low = 0
high = 8

goal = 0.100

meridian = tau/6

P = surface_plot(lambda u, v : (f(u) * cos(v), f(u) * sin(v), u),   (low, high),   (0,tau))


def integrand(s):
    temp = C/f(s) * sqrt((1+ f_prime(s)^2)/(f(s)^2-C^2) )
    if type(temp) is sage.rings.complex_number.ComplexNumber:
        return NaN
    return temp
g = lambda x: numerical_integral(integrand(s) , (0, x))[0]

training_angles = []

for C in ( 0.7501, 1.5001, 2.25001, 2.6001):

    temp = find_angle2(meridian, 0.0001)
    training_angles.append(temp)
    print(temp)

    var('u')
    P += line_plot_points(lambda u: (f(u) * cos(g(u)), f(u) * sin(g(u)), u), (low, high), color = 'red')


print()
print("Deviation of training angles:", deviation(training_angles, goal))


print()
print()

testing_angles = []

for C in ( 1.125001, 1.875001, 2.42501):

    temp = find_angle2(meridian, 0.0001)
    testing_angles.append(temp)
    print(temp)

    var('u')
    P += line_plot_points(lambda u: (f(u) * cos(g(u)), f(u) * sin(g(u)), u), (low, high), color = 'orange')



print()
print("Deviation of testing angles:", deviation(testing_angles, goal))

P += line_plot_points(lambda u: (f(u) * cos(meridian), f(u) * sin(meridian), u), (low, high), color = 'yellow')


P


0.101439099696853
0.102957871586168
0.104756773182231
0.100782703049328

Deviation of training angles: 0.00291802704872498


0.0991216407973180
0.106293254726010
0.100883916049233

Deviation of testing angles: 0.00370395633152270


Graphics3d Object

# Status Update (continued)

But I realized that my optimization algorithm was pretty bad. This is essentially a multidimensional minimization problem, where the inputs are the 5 points that determine the quartic, and the function to be minimized (the loss function) is the deviation between the intersection angles and the target. My optimization algorithm was:

1. Start at a point pretty close to the minimum (i.e. a point in the 5-dimensional configuration space that describes the y-values of the 5 actual points, and thus the polynomial)
2. Choose a random direction to slightly perturb the point
3. If the loss function is smaller at the perturbed point, go there.
4. Repeat

(Note that calls to the loss function are costly for this problem, e.g. 5 seconds each.)

In high dimensions, simply finding a direction in which the function decreases was not easy. After a bit of research, I found a better way: [Nelder and Mead's amoeba algorithm](https://en.wikipedia.org/wiki/Nelder%E2%80%93Mead_method), a standard method of finding a minimum of a multidimensional function. There are much faster methods, but the amoeba is simple and reliable, especially when your loss function may not be very smooth. Of course, finding the global minimum is much harder, but I'll settle for local minimums.

# Legal information:
    
The program I'm going to write is a rough translation from C++ of the implementation described in Chapter 10 of *Numerical Recipes*, which is the same algorithm as in Nelder and Mead's original paper. 

The paper is [here](https://drive.google.com/file/d/1Gea-7rrewR_eBp7ObaM6EQBWVH42AtR3/view?usp=sharing). 
        
The book is [here](https://drive.google.com/file/d/1DegA3G8K0FNy-po5Ntbjh8KjATZRu6oL/view?usp=sharing).
        
I might have broken copyright law by finding these PDFs (on SciHub and on some Russian website), but I don't think I broke it very much (else I wouldn't be writing this). But if this code turns out to be useful for the paper (assuming we get enough results to make a paper), I would feel morally obligated to unbreak copyright law by getting a license for *Numerical Recipes*'s code (about $65). However, it's very likely that someone at DAMTP already has an institutional license for the code, in which case that license would cover us. (After all, the book was published by Cambridge University Press, so I assume Cambridge professors know about it.)

*Numerical Recipes* is a very good book. I found it on the introductory CATAM project's resource list. It's mostly useless for that project because it doesn't have the theoretical results that I need to quote, but it's full of useful practical information. For instance, before I opened the book, I was planning on replacing my random step algorithm with gradient descent, since that's we learned in Optimization. Then I read this on page 515:

> A common beginner’s error is to assume that any reasonable way of incorporating gradient information should 
> be about as good as any other. This line of thought leads to the following *not-very-good* algorithm, the 
> *steepest descent method*


I felt guilty about getting the PDF, so I bought the actual book. (Don't worry: I won't go bankrupt spending my food money on textbooks.)

# How the algorithm works

(The [Wikipedia page](https://en.wikipedia.org/wiki/Nelder%E2%80%93Mead_method) has nice visualizations, but its description of the algorithm might not be totally correct. The [paper](https://drive.google.com/file/d/1Gea-7rrewR_eBp7ObaM6EQBWVH42AtR3/view?usp=sharing) has a flow chart)

We initialize by choosing a n-dimensional simplex, e.g. n+1 points that enclose a nonzero n-volume. We evaluate the loss function f at each of the vertices. Intuitively, the simplex should be large enough so that f is significantly different at different vertices, but small enough that the simplex can comfortably fit into different places in the f's landscape. Don't worry about this too much, since the simplex will grow or shrink as needed.

The simplex is going to crawl downwards across the landscape, like an amoeba. Each iteration works like this:

1. Choose the vertex V with the largest value of f. We want to replace this with a new vertex that doesn't have the largest value of f.
2. Reflect V over the opposite facet to get V', and evaluate f(V'). (In practice you find the centroid C of the n points of the opposite facet, and reflect over that.)
3. If f(V') is smaller than the values of all the n+1 original vertices, then this is a good direction. Form V'' so that V' is the midpoint of V'' and C, i.e. we go twice as far in the good direction. Evaluate f(V''), and replace V with whichever of V' or V'' has a smaller value. End of iteration.
4. If f(V') is smaller than the next-to-largest vertex, then we've accomplished our goal of finding a vertex that is no longer the highest. Replace V with V'. End of iteration.
5. If we've reached this step, then f(V') is larger than the n smallest vertices. Let X be whichever of V and V' is smaller. Since V and V' are on opposite sides of the facet, and they are higher up than every vertex of the facet, the minimum must be close to the facet. Let X' be the midpoint of X and C. Calculate f(X'). Since we moved toward the facet, we expect that f(X') < f(V). If so, replace V with X'. End of iteration.
6. What if it's not true that f(X') < f(V)? That doesn't make sense, so the landscape must look weird, e.g. a hill within the simplex. Find the vertex with the smallest vertex W, and replace every other vertex with the midpoint of itself and W. Thus the simplex gets smaller, so hopefully the landscape looks simpler. End of iteration.

The main time cost here is function evaluation, so I'm not going to bother making the other parts of the code faster.

In [4]:
def amoeba(f, n, initial_point, initial_step, goal = -10^6, will_plot3d = False , x_range = (-1,1), y_range = (-1,1)):
    '''n is the dimension
    initial_point is a numpy array of length n
    f is the loss function. It takes numpy vectors of length n as input
    initial_step is how long the sides of the initial simplex should be 
    goal is a number we want the minimum to go below
    x_range and y_range only determine where the plot is'''
    import numpy as np
    vertices = []
    evals = 0 #How many times we call f
    vertices.append( (initial_point, f(initial_point) ) ) #Store the point and its function value
    evals += 1
    #Now we go in each of the n orthogonal basis directions to get a vertex
    for i in range(n):
        perturbation = np.zeros(n) #the n-dimensional zero vector
        perturbation[i] = initial_step #Give it a direction
        temp_point = initial_point + perturbation
        vertices.append( (temp_point, f(temp_point) ) ) #Add the vertex to the list
        evals += 1
    #Note to self: vertices has n+1 elements, not n! 
    iters = 0
    if will_plot3d:
        aux_f = lambda x,y: f( np.array([x,y]) )
        P = plot3d(aux_f, x_range, y_range, opacity = 1)
    while not should_stop() and vertices[0][1] > goal and iters<1000:
        #Stop if we get a point below the goal, or if the user tells it to stop, or if it's taking too many iterations
        iters += 1
        if will_plot3d:
            poly = Polyhedron( [tuple(vertices[i][0]) + (vertices[i][1],) for i in range(n+1)])
            P+= plot(poly, opacity = 0.25, color = 'yellow')
        vertices, evals = amoeba_iterate(f, n, vertices, evals)
        vertices.sort(key = lambda x: x[1]) #Sort so that we can find the smallest element in the while loop
    if will_plot3d:
        poly = Polyhedron( [tuple(vertices[i][0]) + (vertices[i][1],) for i in range(n+1)])
        P+= plot(poly, opacity = 0.25, color = 'yellow')
    print("Independent variables: ", vertices[0][0])
    print("Dependent variable:", vertices[0][1])
    print("Function evaluations:", evals)
    if will_plot3d:
        show(P)
        
    
    
def amoeba_iterate(f, n, vertices, evals):
    import numpy as np
    vertices.sort(key = lambda x: x[1]) #Sort the vertices from least value of f to greatest
    V, f_V = vertices[n] #Remember the list is indexed from 0 to n
    C = np.zeros(n)
    for i in range(n): #i.e. 0,1,...,n-1
        C += vertices[i][0]
    C = C / n #C is the centroid of the other n points
    V_prime = C + (C-V) #Reflect V over C
    f_V_prime = f(V_prime)
    evals += 1
    if f_V_prime < vertices[0][1]: #V_prime is the best point so far
        V_prime_prime = C + 2* (C-V) #Reflect V over C, twice as far
        f_V_prime_prime = f(V_prime_prime)
        evals += 1
        if f_V_prime_prime < f_V_prime: #Use whichever one is better
            vertices[n] = (V_prime_prime,f_V_prime_prime)
        else:
            vertices[n] = (V_prime,f_V_prime)
        return vertices, evals
    elif f_V_prime < vertices[n-1][1]: #The old second-highest point is now the highest point
        vertices[n] = (V_prime,f_V_prime)
        return vertices, evals
    else: #Reflecting didn't get to a point smaller than the second-highest point
        if f_V_prime < f_V: #X is the better of V and V_prime
            X = V_prime
            f_X = f_V_prime
        else:
            X = V
            f_X = f_V 
        X_prime = (C + X)/2 #Contract X toward the centroid
        f_X_prime = f(X_prime)
        evals += 1
        if f_X_prime < f_V: #Going toward the centroid makes f smaller
            vertices[n] = (X_prime,f_X_prime)
            return vertices, evals
        else: #Going toward the centroid does not make f smaller, which is weird
            #So we move every point halfway towards the lowest point
            L = vertices[0][0] #the lowest point
            for i in range(1,n+1): #1,2,...,n
                Y = vertices[i][0]
                Z = (Y + L) / 2
                vertices[i] = (Z, f(Z) )
                evals += 1
            return vertices, evals

Out of paranoia, I'm going to also test my random step algorithm to make sure it's slower.

In [5]:
def random_step(f, n, initial_point, step_size, goal = -10^6, will_plot3d = False , x_range = (-1,1), y_range = (-1,1)):
    import time
    '''n is the dimension
    initial_point is a numpy vector of length n
    f is a function that takes numpy vectors of length n as input
    step_size is how far the point jumps at each step
    goal is a number we want the minimum to go below
    x_range and y_range only determine where the plot is'''
    import numpy as np
    import random
    evals = 0
    old_point = initial_point.copy()
    old_f = f(old_point)
    evals += 1
    if will_plot3d:
        aux_f = lambda x,y: f( np.array([x,y]) )
        P = plot3d(aux_f, x_range, y_range, opacity = 0.75) 
    iters = 0
    while not should_stop() and old_f > goal and iters<2000:
        iters += 1
        new_point = old_point.copy()
        for i in range(n):
            new_point[i] += (random.random() - 0.5 ) * step_size
            #Perturb the point by a little bit
        new_f = f(new_point)
        evals += 1
        if new_f < old_f:
            if will_plot3d:
                P += line( [ list(old_point) + [old_f]  , list(new_point) + [new_f] ], color = 'red')
  #              show(line( [old_point, new_point], color = 'red'))
  #              time.sleep(1)
   #             show(P)
            old_f = new_f
            old_point = new_point
    print("Independent variables: ", old_point)
    print("Dependent variable:", old_f)
    print("Function evaluations:", evals)
    if will_plot3d:
        show(P)


First we test a paraboloid. It's convex, so finding the minimum is easy.

In [6]:
def paraboloid(two_array):
    return two_array[0]^2 + two_array[1]^2

The random step algorithm depends a lot on what the step size is:

In [7]:
import numpy as np
random_step(paraboloid, 2, np.array([-0.9,-0.9]), 0.6, goal = 0.001)

Independent variables:  [-0.01128125  0.02133646]
Dependent variable: 0.0005825112372141103
Function evaluations: 198


In [8]:
random_step(paraboloid, 2, np.array([-0.9,-0.9]), 0.06, goal = 0.001)

Independent variables:  [-0.00873587  0.00959047]
Dependent variable: 0.00016829247884473167
Function evaluations: 163


In [9]:
random_step(paraboloid, 2, np.array([-0.9,-0.9]), 0.2, goal = 0.001, will_plot3d = True)

Independent variables:  [0.00914183 0.00521566]
Dependent variable: 0.00011077621161857513
Function evaluations: 40


Graphics3d Object

For the amoeba, choosing the wrong initial step size doesn't have a strong negative effect.

In [10]:
amoeba(paraboloid, 2, np.array([-0.9,-0.9]), 0.6, goal = 0.001)

Independent variables:  [-2.22044605e-16 -2.22044605e-16]
Dependent variable: 9.860761315262648e-32
Function evaluations: 5


In [11]:
amoeba(paraboloid, 2, np.array([-0.9,-0.9]), 0.06, goal = 0.001)

Independent variables:  [-0.01863281  0.02332031]
Dependent variable: 0.0008910186767577021
Function evaluations: 29


In [12]:
amoeba(paraboloid, 2, np.array([-0.9,-0.9]), 0.2, will_plot3d = True, goal = 0.001)

Independent variables:  [-0.01710815 -0.00618286]
Dependent variable: 0.0003309167176485136
Function evaluations: 25


Graphics3d Object

Now we use a more difficult function. 
[Rosenbrock's banana function](https://en.wikipedia.org/wiki/Rosenbrock_function) is $ f(x,y) = (1-x)^2 + 100(y-x^2)^2 $, with its minimum at $ f(1,1) = 0 $.
It's easy to find the long, banana-shaped valley, but it's harder to travel along the valley to find
the minimum.

In [13]:
def rosenbrock(two_array):
    return (1-two_array[0])^2 + 100*(two_array[1]-two_array[0]^2)^2

In [14]:
random_step(rosenbrock, 2, np.array([-0.9,0]), 0.6, goal = 0.001)

Independent variables:  [0.99358753 0.98916612]
Dependent variable: 0.0004213447404513201
Function evaluations: 950


In [15]:
random_step(rosenbrock, 2, np.array([-0.9,0]), 0.06, goal = 0.001)

Independent variables:  [0.98358307 0.96893983]
Dependent variable: 0.0004957663246484724
Function evaluations: 1124


In [16]:
random_step(rosenbrock, 2, np.array([-0.9,0]), 0.2, goal = 0.001, will_plot3d = True)

Independent variables:  [0.98673185 0.97320485]
Dependent variable: 0.00019495744718916056
Function evaluations: 784


Graphics3d Object

And now we see that the amoeba method is at least 5 times faster than the random step method.

In [17]:
amoeba(rosenbrock, 2, np.array([-0.9,0]), 0.6, goal = 0.001, )

Independent variables:  [0.97757576 0.95383684]
Dependent variable: 0.0008331841881649162
Function evaluations: 87


In [18]:
amoeba(rosenbrock, 2, np.array([-0.9,0]), 0.06, goal = 0.001, )

Independent variables:  [0.97271333 0.94665762]
Dependent variable: 0.0007682205609687321
Function evaluations: 98


In [19]:
amoeba(rosenbrock, 2, np.array([-0.9,0]), 0.2, goal = 0.001, will_plot3d = True, x_range = (-1,1.1), y_range = (-0.15,1.1))

Independent variables:  [1.00515014 1.00828254]
Dependent variable: 0.000444423281585092
Function evaluations: 76


Graphics3d Object

I expect that the amoeba's advantage will be even greater when there are more dimensions. But I haven't actually tested the amoeba on the "find a quartic that gets the correct geodesic-meridian angles" problem, since that might take another day. Now I have to do actual Optimization homework, which isn't quite as fun as this. I'll probably get back to this on Tuesday.